Code Cell 1 - Import Data and Clean Data

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Paths to data directories
DATA_DIR = "512953-history-standings"
H2H_DIR = "512953-history-teamgamecenter"

# ---------------------------
# Load standings data
# ---------------------------
def load_history_data(data_dir=DATA_DIR):
    all_data = []

    for file in os.listdir(data_dir):
        if file.endswith(".csv"):
            year = file.replace(".csv", "")
            file_path = os.path.join(data_dir, file)
            
            try:
                df = pd.read_csv(file_path, dtype=str)

                if "Year" in df.columns:
                    df = df.drop(columns=["Year"])

                df.insert(0, "Year", int(year))

                if "Record" in df.columns:
                    df["Record"] = (
                        df["Record"]
                        .astype(str)
                        .str.replace("/", "-", regex=False)
                        .str.replace(r"-20\d{2}$", "", regex=True)
                    )
                    df = df.rename(columns={"Record": "RegularSeasonRecord"})

                cols = list(df.columns)
                if "TeamName" in cols and "PlayoffRank" in cols:
                    col_order = ["Year", "TeamName", "PlayoffRank"] + [
                        c for c in cols if c not in ["Year", "TeamName", "PlayoffRank"]
                    ]
                else:
                    col_order = ["Year"] + [c for c in cols if c != "Year"]

                df = df[col_order]

                all_data.append(df)
                print(f"Loaded standings {file}")

            except Exception as e:
                print(f"Error loading {file}: {e}")

    if all_data:
        full_df = pd.concat(all_data, ignore_index=True)
        if "PlayoffRank" in full_df.columns:
            full_df["PlayoffRank"] = pd.to_numeric(full_df["PlayoffRank"], errors="coerce")
            full_df = full_df.sort_values(by=["Year", "PlayoffRank"], ascending=[True, True])
        return full_df
    else:
        raise FileNotFoundError("No standings CSV files found.")


# ---------------------------
# Load head-to-head data
# ---------------------------
def load_h2h_data(base_dir=H2H_DIR):
    all_games = []

    for year in os.listdir(base_dir):
        year_path = os.path.join(base_dir, year)
        if not os.path.isdir(year_path):
            continue

        for file in os.listdir(year_path):
            if file.endswith(".csv"):
                week = file.replace(".csv", "")
                file_path = os.path.join(year_path, file)

                try:
                    df = pd.read_csv(file_path, dtype=str)

                    if {"Owner", "Opponent", "Total", "Opponent Total"}.issubset(df.columns):
                        temp = df[["Owner", "Opponent", "Total", "Opponent Total"]].copy()
                        temp["Year"] = int(year)
                        temp["Week"] = int(week)

                        temp["Total"] = pd.to_numeric(temp["Total"], errors="coerce")
                        temp["Opponent Total"] = pd.to_numeric(temp["Opponent Total"], errors="coerce")

                        all_games.append(temp)
                except Exception as e:
                    print(f"Error loading {file_path}: {e}")

    if all_games:
        return pd.concat(all_games, ignore_index=True)
    else:
        raise FileNotFoundError("No head-to-head matchup files found.")


# ---------------------------
# Build all DataFrames
# ---------------------------
df = load_history_data()
h2h_df = load_h2h_data()

# ---------------------------
# Build finals_summary directly here
# ---------------------------
champions = df[df["PlayoffRank"] == 1][
    ["Year", "ManagerName", "RegularSeasonRecord", "PointsFor", "PointsAgainst"]
].rename(
    columns={
        "ManagerName": "ChampionManager",
        "RegularSeasonRecord": "ChampionRegularSeasonRecord",
        "PointsFor": "ChampionPointsFor",
        "PointsAgainst": "ChampionPointsAgainst"
    }
)

first_losers = df[df["PlayoffRank"] == 2][
    ["Year", "ManagerName", "RegularSeasonRecord", "PointsFor", "PointsAgainst"]
].rename(
    columns={
        "ManagerName": "FirstLoserManager",
        "RegularSeasonRecord": "FirstLoserRegularSeasonRecord",
        "PointsFor": "FirstLoserPointsFor",
        "PointsAgainst": "FirstLoserPointsAgainst"
    }
)

finals_summary = pd.merge(champions, first_losers, on="Year", how="inner").sort_values("Year").reset_index(drop=True)

# ---------------------------
# Derived finishers tables available to all cells
# ---------------------------

# Vectorized per-year max PlayoffRank
_max_rank = df.groupby("Year")["PlayoffRank"].transform("max")

# 1) Last place per year (PlayoffRank == per-year max)
last_place_df = (
    df[df["PlayoffRank"] == _max_rank]
    [["Year", "ManagerName", "TeamName", "PlayoffRank"]]
    .sort_values(["Year", "PlayoffRank"])
    .reset_index(drop=True)
)

# 2) Bottom 3 per year (max, max-1, max-2); guard for small leagues
_cutoff = (_max_rank - 2).clip(lower=1)  # if league <3, this gracefully includes all teams
bottom3_df = (
    df[df["PlayoffRank"] >= _cutoff]
    [["Year", "ManagerName", "TeamName", "PlayoffRank"]]
    .sort_values(["Year", "PlayoffRank"])
    .reset_index(drop=True)
)

# ---------------------------
# Derived finishers tables available to all cells
# ---------------------------

# Vectorized per-year max PlayoffRank
_max_rank = df.groupby("Year")["PlayoffRank"].transform("max")

# 1) Last place per year (PlayoffRank == per-year max)
last_place_df = (
    df[df["PlayoffRank"] == _max_rank]
    [["Year", "ManagerName", "TeamName", "PlayoffRank"]]
    .sort_values(["Year", "PlayoffRank"])
    .reset_index(drop=True)
)

# 2) Bottom 3 per year (max, max-1, max-2); guard for small leagues
_cutoff = (_max_rank - 2).clip(lower=1)  # if league <3, this gracefully includes all teams
bottom3_df = (
    df[df["PlayoffRank"] >= _cutoff]
    [["Year", "ManagerName", "TeamName", "PlayoffRank"]]
    .sort_values(["Year", "PlayoffRank"])
    .reset_index(drop=True)
)

# Determine winner/loser for each game
h2h_df["Winner"] = h2h_df.apply(
    lambda row: row["Owner"] if row["Total"] > row["Opponent Total"] else row["Opponent"], axis=1
)
h2h_df["Loser"] = h2h_df.apply(
    lambda row: row["Opponent"] if row["Total"] > row["Opponent Total"] else row["Owner"], axis=1
)

# Build win/loss matrix (managers vs managers)
h2h_summary = (
    h2h_df.groupby(["Winner", "Loser"])
    .size()
    .unstack(fill_value=0)
    .sort_index(axis=0)
    .sort_index(axis=1)
)



Code Cell 2 - View Entire Standings Dataframe

# Configure pandas display so we can see everything
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Show the full dataset
df


Code Cell 3 - View Head to Head Dataframe

h2h_df

Code Cell 4 - Manger that won the championship and first Loser

In [ ]:
# Count championships and 1st losers
champ_counts = finals_summary["ChampionManager"].value_counts().rename("Championships")
loser_counts = finals_summary["FirstLoserManager"].value_counts().rename("FirstLosers")

# Merge into one DataFrame
champ_loser_summary = pd.concat([champ_counts, loser_counts], axis=1).fillna(0).astype(int)

# Sort by Championships (for plotting consistency)
champ_loser_summary = champ_loser_summary.sort_values("Championships", ascending=False)

# Plot grouped bar chart
ax = champ_loser_summary.plot(
    kind="bar", figsize=(12,6), edgecolor="black"
)

plt.title("Championships vs 1st Loser Finishes by Manager", fontsize=14)
plt.xlabel("Manager", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.xticks(rotation=45, ha="right")
plt.legend(title="Result")

# Add value labels directly on bars
for container in ax.containers:
    ax.bar_label(container, fmt='%d', label_type='edge', fontsize=9)

plt.tight_layout()
plt.show()

champ_loser_summary


Code Cell 5 - Most Top 3 Finishes

In [ ]:
# 1. Seasons played per manager
seasons_played = df.groupby("ManagerName")["Year"].nunique().rename("SeasonsPlayed")

# 2. Top 3 finishes per manager
top3_df = df[df["PlayoffRank"].between(1, 3, inclusive="both")]
top3_counts = top3_df["ManagerName"].value_counts().rename("Top3Finishes")

# 3. Merge into one DataFrame
top3_summary = pd.concat([seasons_played, top3_counts], axis=1).fillna(0)
top3_summary["Top3Finishes"] = top3_summary["Top3Finishes"].astype(int)

# 4. Visualization
plt.figure(figsize=(12,6))
top3_summary["Top3Finishes"].sort_values(ascending=False).plot(
    kind="bar", color="skyblue", edgecolor="black"
)

plt.title("Top 3 Playoff Finishes by Manager (with Seasons Played)", fontsize=14)
plt.xlabel("Manager", fontsize=12)
plt.ylabel("Top 3 Finishes", fontsize=12)
plt.xticks(rotation=45, ha="right")

# Add labels: top 3 finishes / seasons played
for i, (manager, row) in enumerate(top3_summary.sort_values("Top3Finishes", ascending=False).iterrows()):
    plt.text(i, row["Top3Finishes"] + 0.1,
             f"{int(row['Top3Finishes'])}/{int(row['SeasonsPlayed'])}",
             ha="center", va="bottom", fontsize=9)

plt.tight_layout()
plt.show()

top3_summary.sort_values("Top3Finishes", ascending=False)


Code Cell 6 - Most Last Place Finishes

In [ ]:
# 1. Seasons played per manager
seasons_played = df.groupby("ManagerName")["Year"].nunique().rename("SeasonsPlayed")

# 2. Last place finishes per manager
last_place_counts = last_place_df["ManagerName"].value_counts().rename("LastPlaceFinishes")

# 3. Merge into one DataFrame
last_place_summary = pd.concat([seasons_played, last_place_counts], axis=1).fillna(0)
last_place_summary["LastPlaceFinishes"] = last_place_summary["LastPlaceFinishes"].astype(int)

# 4. Visualization
plt.figure(figsize=(12,6))
last_place_summary["LastPlaceFinishes"].sort_values(ascending=False).plot(
    kind="bar", color="salmon", edgecolor="black"
)

plt.title("Last Place Finishes by Manager (with Seasons Played)", fontsize=14)
plt.xlabel("Manager", fontsize=12)
plt.ylabel("Last Place Finishes", fontsize=12)
plt.xticks(rotation=45, ha="right")

# Add labels: last place finishes + seasons played
for i, (manager, row) in enumerate(last_place_summary.sort_values("LastPlaceFinishes", ascending=False).iterrows()):
    plt.text(i, row["LastPlaceFinishes"] + 0.1,
             f"{int(row['LastPlaceFinishes'])}/{int(row['SeasonsPlayed'])}",
             ha="center", va="bottom", fontsize=9)

plt.tight_layout()
plt.show()

last_place_summary.sort_values("LastPlaceFinishes", ascending=False)


Code Cell 7 - Bottom 3 Finishes

In [ ]:
# 1. Seasons played per manager
seasons_played = df.groupby("ManagerName")["Year"].nunique().rename("SeasonsPlayed")

# 2. Bottom 3 finishes per manager
bottom3_counts = bottom3_df["ManagerName"].value_counts().rename("Bottom3Finishes")

# 3. Merge into one DataFrame
bottom3_summary = pd.concat([seasons_played, bottom3_counts], axis=1).fillna(0)
bottom3_summary["Bottom3Finishes"] = bottom3_summary["Bottom3Finishes"].astype(int)

# 4. Visualization
plt.figure(figsize=(12,6))
bottom3_summary["Bottom3Finishes"].sort_values(ascending=False).plot(
    kind="bar", color="orange", edgecolor="black"
)

plt.title("Bottom 3 Finishes by Manager (with Seasons Played)", fontsize=14)
plt.xlabel("Manager", fontsize=12)
plt.ylabel("Bottom 3 Finishes", fontsize=12)
plt.xticks(rotation=45, ha="right")

# Add labels: bottom 3 finishes / seasons played
for i, (manager, row) in enumerate(bottom3_summary.sort_values("Bottom3Finishes", ascending=False).iterrows()):
    plt.text(i, row["Bottom3Finishes"] + 0.1,
             f"{int(row['Bottom3Finishes'])}/{int(row['SeasonsPlayed'])}",
             ha="center", va="bottom", fontsize=9)

plt.tight_layout()
plt.show()

bottom3_summary.sort_values("Bottom3Finishes", ascending=False)


Code Cell 8 - Head2Head Summary

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(h2h_summary, annot=True, fmt="d", cmap="Blues", cbar=False)

plt.title("Head-to-Head Wins Matrix (Rows = Winner, Cols = Loser)", fontsize=14)
plt.xlabel("Loser")
plt.ylabel("Winner")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)

plt.tight_layout()
plt.show()


